<a href="https://colab.research.google.com/github/yaeeun916/RP-Lab-DL-School-HW/blob/main/brain_tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. 사용자 정의 PYTORCH DATALOADER 

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# 경고 메시지 무시하기
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # 반응형 모드 설정

### 데이터셋 다운받아 /data/brain_tumor/ 경로에 위치시키기

참고: 다운받은 뇌종양 데이터셋의 구조

Brain-Tumor-Classification-DataSet-master 폴더 내에 Testing, Training 폴더 내에 glioma_tumor, meningioma_tumor, no_tumor, pituitary_tumor 폴더

glioma_tumor, meningioma_tumor, no_tumor, pituitary_tumor 폴더 내에 이미지 (파일 이름: image(인덱스).jpg)

In [7]:
!wget https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet/archive/master.zip
os.makedirs("/data/brain_tumor/", exist_ok=True)
import zipfile
with zipfile.ZipFile("master.zip","r") as zip_ref:
  zip_ref.extractall("/data/brain_tumor/")
%cd /data/brain_tumor/

--2021-02-17 02:01:27--  https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet/zip/master [following]
--2021-02-17 02:01:28--  https://codeload.github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [                <=> ]  86.97M  25.3MB/s    in 3.6s    

2021-02-17 02:01:31 (24.3 MB/s) - ‘master.zip’ saved [91198591]

/data/brain_tumor


### class로 custom dataset 만들기

기본 구성: 

def __init__(self, x, transform)

def __len__(self)

def __getitem__(self,index)

In [30]:
class BrainTumorDataset(Dataset):
  train_list=["Brain-Tumor-Classification-DataSet-master/Training/glioma_tumor",
              "Brain-Tumor-Classification-DataSet-master/Training/meningioma_tumor",
              "Brain-Tumor-Classification-DataSet-master/Training/no_tumor",
              "Brain-Tumor-Classification-DataSet-master/Training/pituitary_tumor"]
  test_list=["Brain-Tumor-Classification-DataSet-master/Testing/glioma_tumor",
              "Brain-Tumor-Classification-DataSet-master/Testing/meningioma_tumor",
              "Brain-Tumor-Classification-DataSet-master/Testing/no_tumor",
              "Brain-Tumor-Classification-DataSet-master/Testing/pituitary_tumor"]

  def __init__(self, root_dir, train=True, transform=None, target_transform=None):
        #매개변수 :
        #    root_dir (문자열): 모든 이미지가 있는 폴더 경로 (여기선 /data/brain_tumor/)
        #    train: True면 train set, False면 test set
        #    transform (호출가능한 함수, 선택적 매개변수): 샘플에 적용 될 수 있는 선택적 변환

        self.root_dir = root_dir
        self.train = train  
        self.transform = transform
        self.target_transform=target_transform

        if self.train:
          self.train_data=[]
          self.train_label=[]
          for tr in train_list: 
            folder=os.path.join(self.root_dir, tr) #glioma_tumor, meningioma_tumor, no_tumor, pituitary_tumor 폴더의 절대경로
            for currentdir, dirs, files in os.walk(folder): 
              for file in files: #files: folder경로 내 모든 파일
                image_name=os.path.join(folder, file)
                self.train_data.append(image_name)
                self.train_label.append(tr.split('/')[-1]) #glioma_tumor, meningioma_tumor, no_tumor, pituitary_tumor
          else:
            self.test_data=[]
            self.test_label=[]
            for ts in test_list:
              folder=os.path.join(self.root_dir, ts)
              for currentdir, dirs, files in os.walk(folder):
                for file in files:
                  image_name=os.path.join(folder, file)
                  self.test_data.append(image_name)
                  self.test_label.append(ts.split('/'[-1]))


  def __len__(self):
      if self.train:
        return len(self.train_data)
      else:
        return len(self.test_data)

  def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if self.train:
          img, target=io.imread(self.train_data[index]), self.train_label[index]
        else:
          img, target=io.imread(self.test_data[index]), self.test_label[index]
        
        if self.transform:
          img=self.transform(img)
        if self.target_transform:
          target=self.target_transform(target)

        return img, target

